In [9]:
import ase.io
import os
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pickle
import random

In [10]:
k = open("/checkpoint/mshuaibi/mappings/adslab_ref_energies_ocp728k.pkl", "rb")
ref = pickle.load(k)

In [11]:
t = open("/checkpoint/electrocatalysis/relaxations/mapping/pickled_mapping/adslab_ref_energies_full.pkl", "rb")
new = pickle.load(t)

In [12]:
len(list(new.keys()))

1308508

In [ ]:
for i in ref.keys():
    assert ref[i] == new[i]

In [ ]:
model = "ocp20M_2020-08-16-21-53-07-ocp20Mv6_schnet_lr0.0001_ch512_fltr256_gauss200_layrs3"

In [ ]:
j = open("/private/home/mshuaibi/ocp-modeling/utils/val_sample.txt", "r")
paths = j.read().splitlines()
paths = ["/checkpoint/sidgoyal/electro_done/random987347.traj"]
dirname = f"/private/home/mshuaibi/baselines/expts/relaxations/{model}/ml_results/"

for dft_path in paths:
    dft_images = ase.io.read(dft_path, ":")
    dft_energies = [image.get_potential_energy(apply_constraint=False) for image in dft_images]
    randomid = os.path.basename(dft_path)[:-5]
    ml_path = os.path.join(dirname, f"{randomid}/ml_idx_{randomid}.traj")
    ml_images = ase.io.read(ml_path, ":")
    ml_energies = np.array([image.get_potential_energy(apply_constraint=False) + ref[randomid] for image in ml_images])
    random.seed(234562)
    mldft_steps = np.array(random.sample(range(len(ml_images)), 15))
    ml_steps = np.array(range(len(ml_energies)))

    ml_dft_path = os.path.join(dirname, f"{randomid}/dft_validation/ml_idx_{randomid}")
    
    ml_dft_images = []
    ml_dft_steps = []
    for idx, step in enumerate(os.listdir(ml_dft_path)):
        try:
            step_path = os.path.join(ml_dft_path, step)
            image = ase.io.read(os.path.join(step_path, "OUTCAR"))
            ml_dft_images.append(image)
            ml_dft_steps.append(mldft_steps[idx])
        except:
            continue

    ml_dft_energies= np.array([image.get_potential_energy(apply_constraint=False) for image in ml_dft_images])
#     for image in ml_images:
#         image.set_cell([0,0,0])
#     ase.io.write(
#         os.path.join(dirname, f"{randomid}/ml_{randomid}.gif"), 
#         ml_images,
#         rotation="-75x, 45y, 10z", 
#         interval=5
#     )
    

    plt.figure(figsize=(8,8))
    plt.plot(range(len(dft_energies)), dft_energies, label="DFT images", color="black")
    plt.plot(ml_steps, ml_energies, label=f"ML images", color="b")    
    plt.scatter(ml_dft_steps, ml_dft_energies, label="DFT@ML images", color="r")
    plt.xlabel("Step", fontsize=20)
    plt.ylabel("Energy, eV", fontsize=20)
    plt.tick_params(axis="both", labelsize=15)
    plt.legend(fontsize=20)
    plt.title(randomid, fontsize=20)
    plt.tight_layout()
    plt.savefig(f"{dirname}/{randomid}_dft@ml.png")

In [ ]:
ase.io.write("dft_987347.traj", ml_dft_images)

In [ ]:
mlimages = [ml_images[idx] for idx in mldft_steps]
ase.io.write("ml_987347.traj", mlimages)

In [ ]:
ml_dft_images[7].cell

In [ ]:
mlimages[7].cell

In [ ]:
ml_images = ase.io.Trajectory("/private/home/mshuaibi/baselines/expts/relaxations/ml_results/random987347/ml_idx_0_random987347.traj")
dft_images = ase.io.Trajectory("/checkpoint/sidgoyal/electro_done/random987347.traj")

In [ ]:
ase.io.write("ml_987347.gif", ml_images)
ase.io.write("dft_987347.gif", dft_images)